# `gsl::span`

An instance of `span<T>` (formerly `array_view<T>`) denotes a sequence of instances of `T`, modifiable (unless` T` is a `const` type). It is a **non-owner view** in the sense that the instance of `span <T>` is built on top of another collection, and only provides simplified and standardized access to its elements.

## Construction

This type is the equivalent of a pointer and a size, and is based on the assumption that the elements of the sequence are in a contiguous memory area. Therefore, a  `gsl::span<T>` can only be built from collections that respect this memory contiguity, such as `std::vector<T>`, `std::array<T>`, `std::string` or an ordinary array `T[N]`.

In [4]:
%%file tmp.gsl-span.cpp

#include <iostream>
#include <vector>
#include <gsl/gsl>

template <typename T>
void display( gsl::span<T> data )
 {
   for( auto e : data) std::cout << e << ' ';
   std::cout << std::endl;
 }
 
int main()
 {
  std::vector<int> v{ 1, 2, 3, 4, 5 };
  gsl::span<int> s1{ v };
  display(s1);
 
  std::array<int, 5> a{ 10, 20, 30, 40, 50 };
  gsl::span<int> s2{ a };
  display(s2);
 
  int arr[] = { 100, 200, 300, 400, 500 } ;
  gsl::span<int> s3{ arr } ;
  display(s3);
    
  return 0 ;
}

Overwriting tmp.gsl-span.cpp


In [5]:
!rm -f tmp.gsl-span.exe && g++ -std=c++17 -I./ tmp.gsl-span.cpp -o tmp.gsl-span.exe

In [6]:
!./tmp.gsl-span.exe

1 2 3 4 5 
10 20 30 40 50 
100 200 300 400 500 


## Static size

In the examples above, the size of the view is determined at the runtime. When the size is known in advance, it is recommended to specify it explicitly :

In [7]:
%%file tmp.gsl-span.cpp

#include <iostream>
#include <gsl/gsl>

template <typename T, std::ptrdiff_t size>
void display( gsl::span<T,size> data )
 {
   for( auto e : data)
     std::cout << e << ' ';
   std::cout << std::endl;
 }
 
int main()
 {
  std::array<int,5> data { 1, 2, 3, 4, 5 } ;
  gsl::span<int,5> s{ data };
  display(s);
    
  return 0 ;
}

Overwriting tmp.gsl-span.cpp


In [8]:
!rm -f tmp.gsl-span.exe && g++ -std=c++17 -I./ tmp.gsl-span.cpp -o tmp.gsl-span.exe

In [9]:
!./tmp.gsl-span.exe

1 2 3 4 5 


In fact, when the second parameter is not specified, it receives the default value `-1`, which means `dynamic_extent`.

## Sub views

A view portion can be extracted from the start, from the end, or between a start and end index (excluded). This is applied at runtime, or at compilation time if the indices are already known.

In [10]:
%%file tmp.gsl-span.cpp

#include <iostream>
#include <gsl/gsl>

template <typename T, std::ptrdiff_t size>
void display( gsl::span<T,size> data )
 {
   for( auto e : data) std::cout << e << ' ';
   std::cout << std::endl;
 }
 
int main()
 {
  std::array<int,5> data { 1, 2, 3, 4, 5 } ;
  gsl::span<int,5> s{ data } ;

  // run time
  display(s.first(2)) ;
  display(s.last(2)) ;
  display(s.subspan(1,3)) ;
    
  // compile time
  display(s.first<2>()) ;
  display(s.last<2>()) ;
  display(s.subspan<1,3>()) ;
    
  return 0 ;
}

Overwriting tmp.gsl-span.cpp


In [11]:
!rm -f tmp.gsl-span.exe && g++ -std=c++17 -I./ tmp.gsl-span.cpp -o tmp.gsl-span.exe

In [12]:
!./tmp.gsl-span.exe

1 2 
4 5 
2 3 4 
1 2 
4 5 
2 3 4 


## To remember

As Etienne Boespflug puts it very well :

> Wherever you could use a pointer and a size, prefer `gsl::span`, which simplifies passing arguments (`(gsl::span<T> s)` rather than `(T t [], std::size_t size)`) and responds to the interface of a container, which avoids errors due to pointer arithmetic.

> From a semantic point of view, it makes it clear that you are only working on a view rather than a pointer or container that owns the resource.

> Being just a view, it avoids copying, so consider using `gsl::span<T>` instead of a `std::vector<T> const &`. This will produce more flexible code.

> `gsl::span` does not perform bound checking, like the much contested `std::vector::at`, but allows the addition of debug information for out-of-bounds accesses from the part of the implementation.

> Using `gsl::span` also allows you to write generic code, which will benefit from a simple interface and which will be usable with a simple pointer/size pair, std::vector, std::array, std::string, C style arrays or even boost::container::vector, QVector, etc… So don't hesitate to overuse them.

## Some details

* A view will not be able to be constructed from an instance of `initializer_list`, since it is a collection of an ephemeral nature, whereas a view must rely on a durable underlying structure.
* The C++20 standard should contain a `std::span` like the` gsl::span`.
* There was some hunger for multi-dimensional views, which Microsoft referred to as `multi_span`, and which others call `mdspan`. The proposal [P0009](https://wg21.link/p0009) has been submitted, discussed [here](https://cppcast.com/bryce-lelbach-mdspan/). The [reference implementation](https://github.com/kokkos/mdspan) is provided within Kokkos, and discussed [here](https://ieeexplore.ieee.org/document/8945643). Another implementation within Magnum is discussed [here](https://blog.magnum.graphics/backstage/multidimensional-strided-array-views).

## Exercise

Write your own `my_span<T>` :
1. Provide a constructor from a `std::array<T>`.
2. Add `size()` and `operator[]` member functions, which should make the code compilable.
3. Try turning the constructor into a pattern, assuming that the object passed to the constructor still has `size()` and `operator[]` member functions.

In [ ]:
%%file tmp.gsl-span.cpp

#include <iostream>
#include <array>
#include <cassert>

// ... my_span ..

template <typename T>
void display( my_span<T> data )
 {
   for( std::size_t i = 0 ; i < data.size() ; ++i )
     std::cout << data[i] << ' ' ;
   std::cout << std::endl ;
 }
 
int main()
 {
  std::array<int,5> data { 1, 2, 3, 4, 5 } ;
  my_span<int> s { data } ;
  display(s) ;  
  return 0 ;
}

In [ ]:
!rm -f tmp.gsl-span.exe && g++ -std=c++17 -I./ tmp.gsl-span.cpp -o tmp.gsl-span.exe

In [ ]:
!./tmp.gsl-span.exe

## References

* https://etienne-boespflug.fr/cpp/257-c20-stdspan/
* https://github.com/microsoft/GSL/blob/master/include/gsl/span
* http://codexpert.ro/blog/2016/03/07/guidelines-support-library-review-spant/
* http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2019/p0009r9.html
* https://github.com/kokkos/mdspan

© *CNRS 2020*  
*Assembled and written in french by David Chamont, translated by Karim Asnaoui, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)